In [ ]:
print("Ok")

In [ ]:
%pwd

In [ ]:
import os
os.chdir("../")

In [ ]:
%pwd

In [ ]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [ ]:
def load_pdf_files(data):
    loader = DirectoryLoader(
        data, 
        glob="*.pdf", 
        loader_cls=PyPDFLoader
    )
    documents = loader.load()
    return documents
    

In [ ]:
extracted = load_pdf_files("data")

In [ ]:
extracted

In [ ]:
len(extracted)

In [ ]:
from typing import List
from langchain.schema import Document
def filter_to_minimal_docs(docs: List[Document])-> List[Document]:
    minimal_docs: List[Document]=[]
    for doc in docs:
        src = doc.metadata.get("source")
        minimal_docs.append(
            Document(
                page_content=doc.page_content,
                metadata={"source": src}
            )   
        )  
    return minimal_docs

In [ ]:
minimal_docs=filter_to_minimal_docs(extracted)

In [ ]:
minimal_docs

In [ ]:
def text_split(minimal_docs):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=500,
        chunk_overlap=20,
    )
    texts_chunk = text_splitter.split_documents(minimal_docs)
    return texts_chunk

In [ ]:
texts_chunk=text_split(minimal_docs)
print(f"Number of chunks: {len(texts_chunk)}")

In [ ]:
from langchain_huggingface import HuggingFaceEmbeddings
def download_embeddings():
    model_name="sentence-transformers/all-MiniLM-L6-v2"
    embeddings= HuggingFaceEmbeddings(
        model_name=model_name,
    )
    return embeddings
embedding=download_embeddings()

In [ ]:
vector = embedding.embed_query("Hello world")
vector

In [ ]:
print("Vector length:", len(vector))

In [ ]:
from dotenv import load_dotenv
import os
load_dotenv()

In [ ]:
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

In [ ]:
pc

In [ ]:
from pinecone import Pinecone, ServerlessSpec

# Initialize Pinecone client
pc = Pinecone(api_key="pcsk_9wB4M_4ttZHQPrkuFoDkUv7zB8dtrAKNwN7hdzQhqWdgpNUT8PLW5M6ptX3D3oXgpuyCS") 

index_name = "medical-chatbot"
if not pc.has_index(index_name):
    pc.create_index(
        name=index_name,
        dimension=384,
        metric="cosine",
        # Change the region to one supported by the free plan, like 'us-east-1'
        spec=ServerlessSpec(cloud="aws", region="us-east-1")
    )
index = pc.Index(index_name)

In [ ]:
from pinecone import Pinecone
from langchain_pinecone import PineconeVectorStore

pc = Pinecone(api_key="YOUR_API_KEY")

index_name = "medical-chatbot"
index = pc.Index(index_name)

docsearch = PineconeVectorStore(
    index=index,
    embedding=embedding,
    text_key="text"   # make sure your docs have "text" key
)

docsearch.add_documents(texts_chunk)


In [ ]:
from pinecone import Pinecone

pc = Pinecone(api_key="pcsk_9wB4M_4ttZHQPrkuFoDkUv7zB8dtrAKNwN7hdzQhqWdgpNUT8PLW5M6ptX3D3oXgpuyCS")  # replace with your key


In [ ]:
import os
from pinecone import Pinecone

pc = Pinecone(api_key=os.getenv("PINECONE_API_KEY"))
print("Key loaded:", os.getenv("PINECONE_API_KEY") is not None)


In [ ]:
print("Key loaded:", os.getenv("PINECONE_API_KEY") is not None)


In [ ]:
from pinecone import Pinecone

pc = Pinecone(api_key=os.getenv("PINECONE_API_KEY"))

index_name = "medical-chatbot"

index = pc.Index(index_name)
print("Connected to:", index_name)


In [ ]:
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore(
    index=index,
    embedding=embedding,
    text_key="text"
)

docsearch.add_documents(texts_chunk)


In [ ]:
from langchain_pinecone import PineconeVectorStore
docsearch=PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embedding
)

In [ ]:
dswith=Document(
    page_content="dswith is a channel",
    metadata={"source": "youtube"}

)

In [ ]:
docsearch.add_documents(documents=[dswith])

In [ ]:
retriever= docsearch.as_retriever(search_type="similarity", search_kwargs={"k":3})

In [ ]:
retrieved_docs = retriever.invoke("What is acne? ")
retrieved_docs

In [ ]:
from langchain_openai import ChatOpenAI
chatModel = ChatOpenAI(model="gpt-4o")

In [ ]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

In [ ]:
question_answer_chain=create_stuff_documents_chain(chatModel,prompt)
rag_chain=create_retrieval_chain(retriever,question_answer_chain)


In [ ]:
import os
os.environ["GROQ_API_KEY"] = "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"


In [ ]:
import os
from langchain_groq import ChatGroq

# set your real Groq API key
os.environ["GROQ_API_KEY"] = "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"

# initialize using a valid model
llm = ChatGroq(model="llama-3.3-70b-versatile", temperature=0)

# test
response = llm.invoke("What is the treatment of Acne?")
print(response.content)


In [ ]:
response = llm.invoke("What is the treatment of Acne?")
print(response.content)


In [ ]:
response = llm.invoke("How to cure headache?")
print(response.content)

In [ ]:
from langchain_groq import ChatGroq

llm = ChatGroq(model="groq-chat-1.0", temperature=0)


In [ ]:
from langchain_groq import ChatGroq

llm = ChatGroq(model="groq-chat-latest", temperature=0)

response = llm.invoke("What are the symptoms of diabetes?")
print(response.content)


In [ ]:
from pinecone import Pinecone

pc = Pinecone(api_key=PINECONE_API_KEY)
index = pc.Index("medical-chatbot")

print(index.describe_index_stats())
